## Individual Assignment with FeatureTools

Leverage Featuretools, a Python library, to perform automated feature engineering on a sample e-commerce dataset. Use the insights gained through this process to design an optimized data model for a Data Warehouse.

Dataset: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

In [24]:
import pandas as pd
import numpy as np
import featuretools as ft

import uuid

### Read data

In [25]:
customer_df = pd.read_csv('Dataset/olist_customers_dataset.csv')
order_items_df = pd.read_csv('Dataset/olist_order_items_dataset.csv')
order_payments_df = pd.read_csv('Dataset/olist_order_payments_dataset.csv')
orders_df = pd.read_csv('Dataset/olist_orders_dataset.csv')
products_df = pd.read_csv('Dataset/olist_products_dataset.csv')
seller_df = pd.read_csv('Dataset/olist_sellers_dataset.csv')

### Data Cleaning and Prreprocessing

#### Merge order table with order_items table via inner join 'order_id'

In [26]:
orders_df = pd.merge(orders_df, order_items_df, on='order_id', how='inner')

# Drop order_item_id as it is not necessary
orders_df.drop('order_item_id', axis=1, inplace=True)
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


#### Generate unique ID for payment_df

In [27]:
# Function to generate unique IDs using UUID
def generate_unique_id():
    return str(uuid.uuid4())

# Add a unique_id column to DataFrame
order_payments_df['payment_id'] = [generate_unique_id() for _ in range(len(order_payments_df))]



In [28]:
# Drop payment_sequential as it is not necessary
order_payments_df.drop('payment_sequential', axis=1, inplace=True)

In [29]:
order_payments_df.head()

,order_id,payment_type,payment_installments,payment_value,payment_id
0,b81ef226f3fe1789b1e8b2acac839d17,credit_card,8,99.33,3ab81ed0-5e2c-4108-bd60-a5254d2b3632
1,a9810da82917af2d9aefd1278f1dcfa0,credit_card,1,24.39,a7e3d59c-4095-46a8-a310-0c40020587d1
2,25e8ea4e93396b6fa0d3dd708e76c1bd,credit_card,1,65.71,2a6f0a70-e57c-45a5-95e0-81d5489cb7db
3,ba78997921bbcdc1373bb41e913ab953,credit_card,8,107.78,db7a6878-8c70-43a9-b6a7-0014a69e34a0
4,42fdf880ba16b47b59251dd489d4441a,credit_card,2,128.45,4e31d6e3-a9c1-4e6b-81a2-1134b7d7f962


#### Add foreign keys (payment_id) into the fact table (orders_df)

In [30]:
# Merge 'payment_id' from order_payments_df to orders_df based on 'order_id'
orders_df = pd.merge(orders_df, order_payments_df[['order_id', 'payment_id']], on='order_id', how='inner')

In [31]:
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,seller_id,shipping_limit_date,price,freight_value,payment_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,6ac7aff7-d045-4736-965a-d8eb244478f8
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,2980d41c-45e6-44d0-b319-92b8fad690ff
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,50d41bcb-36f3-4ef1-9667-2acc0279fe23
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,21067a95-7bb1-4c0d-a426-8250559317c8
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,7d093c0f-31f5-4cf7-9716-fe16f1ff68b0


Drop order_id at payment table

In [32]:
order_payments_df.drop('order_id', axis=1, inplace=True)

In [33]:
order_payments_df.head()

,payment_type,payment_installments,payment_value,payment_id
0,credit_card,8,99.33,3ab81ed0-5e2c-4108-bd60-a5254d2b3632
1,credit_card,1,24.39,a7e3d59c-4095-46a8-a310-0c40020587d1
2,credit_card,1,65.71,2a6f0a70-e57c-45a5-95e0-81d5489cb7db
3,credit_card,8,107.78,db7a6878-8c70-43a9-b6a7-0014a69e34a0
4,credit_card,2,128.45,4e31d6e3-a9c1-4e6b-81a2-1134b7d7f962


#### Remove duplicates

In [34]:
customer_df = customer_df.drop_duplicates(subset=['customer_id'])
order_items_prod_df = order_items_df.drop_duplicates(subset=['product_id'])
order_payments_df = order_payments_df.drop_duplicates(subset=['payment_id'])
orders_df = orders_df.drop_duplicates(subset=['order_id'])
products_df = products_df.drop_duplicates(subset=['product_id'])


Check column if it is unique

In [35]:
is_unique = ~seller_df['seller_id'].duplicated().any()

if is_unique:
    print("The column is unique.")
else:
    print("The column contains duplicate values.")

The column is unique.


#### Remove NA values

In [36]:
customer_df = customer_df.dropna()
order_payments_df = order_payments_df.dropna()
orders_df = orders_df.dropna()
products_df = products_df.dropna()
seller_df = seller_df.dropna()

In [37]:
# Check for null values column-wise and get the count
lst = [customer_df, order_payments_df, orders_df, products_df, seller_df] #products_df, order_items_df
isnull = []

for item in lst:
    null_counts = item.isnull().sum()
    isnull.append(null_counts)

print(isnull)

[customer_id                 0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64, payment_type            0
payment_installments    0
payment_value           0
payment_id              0
dtype: int64, order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
product_id                       0
seller_id                        0
shipping_limit_date              0
price                            0
freight_value                    0
payment_id                       0
dtype: int64, product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm  

### Print head of df as PNG for report

In [38]:
# # Dictionary of DataFrames
# dataframes = {
#     'customer_df': customer_df,
#     'order_payments_df': order_payments_df,
#     'orders_df': orders_df,
#     'products_df': products_df,
#     'seller_df': seller_df
# }

# # Loop through each DataFrame and print its head()
# for df_name, df in dataframes.items():
#     print(f"DataFrame: {df_name}")
#     print(df.head())
#     print("\n" + "=" * 50 + "\n")  # Separate each DataFrame's head with a line

In [39]:
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_id,seller_id,shipping_limit_date,price,freight_value,payment_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,6ac7aff7-d045-4736-965a-d8eb244478f8
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,21067a95-7bb1-4c0d-a426-8250559317c8
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,7d093c0f-31f5-4cf7-9716-fe16f1ff68b0
5,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,d034f7f9-60db-4728-93ce-a613176a6261
6,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,8395dc6c-9fbb-4720-8d06-dc31a23335c8


In [40]:
products_df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [41]:
customer_df.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP


In [42]:
order_payments_df.head()

,payment_type,payment_installments,payment_value,payment_id
0,credit_card,8,99.33,3ab81ed0-5e2c-4108-bd60-a5254d2b3632
1,credit_card,1,24.39,a7e3d59c-4095-46a8-a310-0c40020587d1
2,credit_card,1,65.71,2a6f0a70-e57c-45a5-95e0-81d5489cb7db
3,credit_card,8,107.78,db7a6878-8c70-43a9-b6a7-0014a69e34a0
4,credit_card,2,128.45,4e31d6e3-a9c1-4e6b-81a2-1134b7d7f962


In [43]:
seller_df.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


### Create and define Entity Set

In [44]:
es = ft.EntitySet(id= 'ecommerce_data')

In [45]:
# Define entities 

es.add_dataframe(dataframe_name='customer_en', dataframe=customer_df, index='customer_id')
es.add_dataframe(dataframe_name='order_payments_en', dataframe=order_payments_df, index='payment_id')
es.add_dataframe(dataframe_name='orders_en', dataframe=orders_df, index='order_id')
es.add_dataframe(dataframe_name='products_en', dataframe=products_df, index='product_id')
es.add_dataframe(dataframe_name='seller_en', dataframe=seller_df, index='seller_id')

C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\inference_functions.py:27: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pdtypes.is_categorical_dtype(series.dtype):
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pd.to_datetime(
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\wo

C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pd.to_datetime(
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argume

C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\inference_functions.py:27: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pdtypes.is_categorical_dtype(series.dtype):
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pd.to_datetime(
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argume

C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\inference_functions.py:27: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pdtypes.is_categorical_dtype(series.dtype):
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pd.to_datetime(
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argume

Entityset: ecommerce_data
  DataFrames:
    customer_en [Rows: 99441, Columns: 4]
    order_payments_en [Rows: 103886, Columns: 4]
    orders_en [Rows: 96460, Columns: 14]
    products_en [Rows: 32340, Columns: 9]
    seller_en [Rows: 3095, Columns: 4]
  Relationships:
    No relationships

### Define and create relationship between entities

In [46]:
# Define relationships
relationships = [('customer_en', 'customer_id', 'orders_en', 'customer_id'),
                 ('order_payments_en', 'payment_id', 'orders_en', 'payment_id'),
                 ('products_en', 'product_id', 'orders_en','product_id'),
                 ('seller_en', 'seller_id', 'orders_en', 'seller_id')]

# Add relationships to EntitySet
for relationship in relationships:
    es.add_relationship(relationship[0], relationship[1], relationship[2], relationship[3])

C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\featuretools\entityset\entityset.py:414: UserWarning: Logical type Categorical for child column seller_id does not match parent column seller_id logical type Unknown. Changing child logical type to match parent.
  warnings.warn(


In [47]:
print(es)

Entityset: ecommerce_data
  DataFrames:
    customer_en [Rows: 99441, Columns: 4]
    order_payments_en [Rows: 103886, Columns: 4]
    orders_en [Rows: 96460, Columns: 14]
    products_en [Rows: 32340, Columns: 9]
    seller_en [Rows: 3095, Columns: 4]
  Relationships:
    orders_en.customer_id -> customer_en.customer_id
    orders_en.payment_id -> order_payments_en.payment_id
    orders_en.product_id -> products_en.product_id
    orders_en.seller_id -> seller_en.seller_id


### Perform Deep Feature Synthesis (DFS)

In [48]:
feature_matrix, feature_defs = ft.dfs(entityset=es, 
                                      target_dataframe_name='orders_en')

# feature_matrix_withAggTrans, feature_defs = ft.dfs(entityset=es, 
#                                       target_dataframe_name='orders_en',
#                                       agg_primitives=['count', 'sum'], 
#                                       trans_primitives=['month', 'year'])

# Display the generated features
# print("Generated Features:")
# print(feature_matrix.head())

C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:825: FutureWarning: The provided callable <function std at 0x000001DE3650C310> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  to_merge = base_frame.groupby(
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:825: FutureWarning: The provided callable <function amax at 0x000001DE36508550> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  to_merge = base_frame.groupby(
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:825: FutureWarning: The provided callable <function mean at 0x000001DE3650C160> is c

C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:839: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pdtypes.is_categorical_dtype(frame.index):
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\logical_types.py:684: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pdtypes.is_categorical_dtype(typed_ser.dtype):
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\logical_types.py:684: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pdtypes.is_categorical_dtype(typed_ser.dtype):
C:\Users\Janice\anaconda3\envs\dmenv\lib\site-packages\woodwork\logical_types.py:684: FutureWarning: is_categorical_dtype is deprecated a

In [49]:
feature_matrix.head()

,order_status,price,freight_value,DAY(order_approved_at),DAY(order_delivered_carrier_date),DAY(order_delivered_customer_date),DAY(order_estimated_delivery_date),DAY(order_purchase_timestamp),DAY(shipping_limit_date),MONTH(order_approved_at),...,seller_en.MIN(orders_en.freight_value),seller_en.MIN(orders_en.price),seller_en.MODE(orders_en.order_status),seller_en.NUM_UNIQUE(orders_en.order_status),seller_en.SKEW(orders_en.freight_value),seller_en.SKEW(orders_en.price),seller_en.STD(orders_en.freight_value),seller_en.STD(orders_en.price),seller_en.SUM(orders_en.freight_value),seller_en.SUM(orders_en.price)
order_id,,,,,,,,,,,,,,,,,,,,,
e481f51cbdc54678b7cc49136f2d6af7,delivered,29.99,8.72,2,4,10,18,2,6,10,...,7.78,24.90,delivered,1,0.352062,0.711674,3.624728,13.581775,619.67,2303.14
53cdb2fc8bc7dce0b6741e2150273451,delivered,118.70,22.76,26,26,7,13,24,30,7,...,8.01,28.89,delivered,1,0.921310,-2.772329,4.691781,21.721685,1718.24,11667.08
47770eb9100c2d0c44946d9cf07ec65d,delivered,159.90,19.22,8,8,17,4,8,13,8,...,0.00,54.90,delivered,1,1.583608,3.082374,8.092765,121.878395,19473.67,221851.45
949d5b44dbf5de918fe9c16f97b45f8a,delivered,45.00,27.20,18,22,2,15,18,23,11,...,2.81,15.00,delivered,1,2.209050,-0.991545,7.973377,27.149642,3082.02,13747.30
ad21c59c0840e6cb83a9ceb5573f8159,delivered,19.90,8.72,13,14,16,26,13,19,2,...,7.39,4.50,delivered,1,2.561087,1.736877,7.019221,32.372457,1823.40,5013.46


In [51]:
for column in feature_matrix.columns:
    print(column)

order_status
price
freight_value
DAY(order_approved_at)
DAY(order_delivered_carrier_date)
DAY(order_delivered_customer_date)
DAY(order_estimated_delivery_date)
DAY(order_purchase_timestamp)
DAY(shipping_limit_date)
MONTH(order_approved_at)
MONTH(order_delivered_carrier_date)
MONTH(order_delivered_customer_date)
MONTH(order_estimated_delivery_date)
MONTH(order_purchase_timestamp)
MONTH(shipping_limit_date)
WEEKDAY(order_approved_at)
WEEKDAY(order_delivered_carrier_date)
WEEKDAY(order_delivered_customer_date)
WEEKDAY(order_estimated_delivery_date)
WEEKDAY(order_purchase_timestamp)
WEEKDAY(shipping_limit_date)
YEAR(order_approved_at)
YEAR(order_delivered_carrier_date)
YEAR(order_delivered_customer_date)
YEAR(order_estimated_delivery_date)
YEAR(order_purchase_timestamp)
YEAR(shipping_limit_date)
customer_en.customer_zip_code_prefix
customer_en.customer_city
customer_en.customer_state
order_payments_en.payment_type
order_payments_en.payment_installments
order_payments_en.payment_value
produ

### Save the feature_matrix as a CSV file

In [34]:
feature_matrix.to_csv('feature_matrix2.csv', index=False)
# feature_matrix_withAggTrans.to_csv('feature_matrix_withAggTrans.csv', index=False)